In [5]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

--2022-11-05 15:18:10--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.186.144, 142.250.185.80, 216.58.212.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.186.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.09s   

2022-11-05 15:18:10 (27.2 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [6]:
import tensorflow as tf
import os
import zipfile

In [7]:
DESIRED_ACCURACY = 0.999

In [8]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("tmp/h-or-s")
zip_ref.close()

In [9]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>DESIRED_ACCURACY):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

In [10]:
callbacks = myCallback()

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150,3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
from tensorflow.keras.optimizers import RMSprop

In [16]:
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(learning_rate=0.001),
             metrics=['accuracy'])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    "tmp/h-or-s",
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [20]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/15
8/8 [==============================] - 2s 105ms/step - loss: 3.5220 - accuracy: 0.6000
Epoch 2/15
8/8 [==============================] - 1s 101ms/step - loss: 0.7251 - accuracy: 0.5750
Epoch 3/15
8/8 [==============================] - 1s 100ms/step - loss: 0.5280 - accuracy: 0.7875
Epoch 4/15
8/8 [==============================] - 1s 99ms/step - loss: 0.4315 - accuracy: 0.8125
Epoch 5/15
8/8 [==============================] - 1s 95ms/step - loss: 0.2233 - accuracy: 0.9875
Epoch 6/15
8/8 [==============================] - 1s 95ms/step - loss: 0.4546 - accuracy: 0.8125
Epoch 7/15
8/8 [==============================] - 1s 91ms/step - loss: 0.1934 - accuracy: 0.9125
Epoch 8/15
8/8 [==============================] - 1s 94ms/step - loss: 0.1361 - accuracy: 0.9250
Epoch 9/15
8/8 [==============================] - 1s 100ms/step - loss: 0.1705 - accuracy: 0.9250
Epoch 10/15
8/8 [==============================] - 1s 92ms/step - loss: 0.0981 - accuracy: 0.9750
Epoch 11/15
8/8 [========